In [ ]:
import pandas as pd

In [ ]:
!pip install huggingface_hub ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
splits = {'train': 'spam_messages_train.csv', 'validation': 'spam_messages_val.csv', 'test': 'spam_messages_test.csv'}
df = pd.read_csv("hf://datasets/mshenoda/spam-messages/" + splits["train"])
df_val = pd.read_csv("hf://datasets/mshenoda/spam-messages/" + splits["validation"])
df_test = pd.read_csv("hf://datasets/mshenoda/spam-messages/" + splits["test"])

In [ ]:
df

,label,text
0,ham,"Funny fact Nobody teaches volcanoes 2 erupt, t..."
1,ham,I sent my scores to sophas and i had to do sec...
2,spam,We know someone who you know that fancies you....
3,ham,Only if you promise your getting out as SOON a...
4,spam,Congratulations ur awarded either �500 of CD g...
...,...,...
47387,ham,please change the sitara tickets dating back t...
47388,spam,from mr . silas sankoh ecobank benin rue du go...
47389,ham,just to get things stirred up : the estate wil...
47390,ham,sorry for the oversight . bill - - - - - origi...


**Zbuduj prosty detektor antyspamowy (klasyfikator)**

Opcje:
  1. klasyfikator oparty na statystycznym podobieństwie, np. TFIDF, odległości Levenhsteina lub występowaniu podobnych wzorców
  2. klasyfikator oparty na ML lub NN, przetwarzający cechy wyciągnięte z tekstu takie jak np. długość, liczba stop wordsów, zawieranie konkretnych słów itp.
  3. klasyfikator w postaci sieci neuronowej przetwarzającej embeddingi
  4. klasyfikator oparty na decyzjach dużego modelu językowego

Metryki:
  1. accuracy
  2. recall
  3. F1
  4. AUC

In [ ]:
from nltk.stem import PorterStemmer
from nltk.downloader import download
from nltk.corpus import stopwords
import re

stemmer = PorterStemmer()
download('stopwords')

def process_text(text):
    text = re.sub(r'<.*?>|[^a-zA-Z\s]', '', text.lower())
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    return ' '.join(stemmed_words)

df2 = pd.DataFrame(df)
df2['text'] = df['text'].apply(process_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np
import pandas as pd # Import pandas
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if len(tf.config.experimental.list_physical_devices('GPU')) > 0:
    print("GPU is available and detected by TensorFlow.")
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)
else:
    print("No GPU detected. Running on CPU.")


df = df2
if len(df) > 1000:
    df = df.sample(n=1000, random_state=42) # random_state for reproducibility
    print(f"Sampled down to {len(df)} records.")
else:
    print(f"Dataset has {len(df)} records, which is less than 10,000. Using full dataset.")

# Map string labels to integers
label_mapping = {
    'ham': 0,
    'spam': 1
}

df['label'] = df['label'].map(label_mapping)


if df['label'].isnull().any():
    print("Warning: Some labels were not mapped. Check 'label_mapping'.")
    print(df[df['label'].isnull()])
    df.dropna(subset=['label'], inplace=True)

df['label'] = df['label'].astype(int)


texts = df['text'].tolist()
labels = df['label'].tolist()
num_labels = len(df['label'].unique())

print(f"Number of unique labels: {num_labels}")


X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=42, stratify=labels)


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_sentences(texts, max_length=512): # Increased max_length for potentially longer reviews
    encoding = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
    return encoding


train_encodings = encode_sentences(X_train)
test_encodings = encode_sentences(X_test)


y_train = np.array(y_train)
y_test = np.array(y_test)

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)


model.compile(optimizer="adamw",
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs = 4

batch_size = 8

train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask']},
    y_train
)).shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']},
    y_test
)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

print(f"\nStarting BERT model training for {epochs} epochs...")
history = model.fit(train_dataset, epochs=epochs, verbose=1)

# --- 7. Prediction and Evaluation ---
print("\nPredicting on test set...")
y_pred_prob = model.predict(test_dataset).logits
y_pred = np.argmax(y_pred_prob, axis=1)

print("\nBERT Classification Report:")

# print(classification_report(y_test, y_pred, zero_division=1, target_names=target_names))
print(classification_report(y_test, y_pred, zero_division=1))

print("\nBERT Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


loss, accuracy = model.evaluate(test_dataset)
print(f"\nTest Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")



print("\nGenerating training visualizations...")

history_dict = history.history


acc = history_dict['accuracy']

val_acc = history_dict.get('val_accuracy')
loss = history_dict['loss']
val_loss = history_dict.get('val_loss')

epochs_range = range(1, len(acc) + 1)


plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
if val_acc:
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)


plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
if val_loss:
    plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.tight_layout()
plt.show()

Num GPUs Available:  0
No GPU detected. Running on CPU.
Sampled down to 1000 records.
Number of unique labels: 2


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


TypeError: 'builtins.safe_open' object is not iterable

In [ ]:
from sklearn.metrics import recall_score, f1_score, roc_auc_score

# Calculate Recall
recall = recall_score(y_test, y_pred)
print(f"\nBERT Recall: {recall:.4f}")

# Calculate F1-score
f1 = f1_score(y_test, y_pred)
print(f"\nBERT F1-score: {f1:.4f}")

try:
    auc = roc_auc_score(y_test, y_pred_prob[:, 1])
    print(f"\nBERT AUC: {auc:.4f}")
except IndexError:
    print("\nCould not calculate AUC. Make sure y_pred_prob has probabilities for both classes.")
except ValueError as e:
    print(f"\nCould not calculate AUC. Error: {e}")
    print("Ensure y_test contains both 0 and 1, and y_pred_prob contains probabilities.")

NameError: name 'y_test' is not defined